# 1. Imports of necessary libraries

In [13]:
import keras #High level interface of TensorFlow
from keras.models import Sequential
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator,load_img,img_to_array
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.utils.np_utils import to_categorical 
import numpy as np 
import matplotlib.pyplot as plt
from PIL import Image
from keras.optimizers import RMSprop
import tensorflowjs as tfjs
import pandas as pd
from sklearn.model_selection import train_test_split

# 2.Data importing& preprocessing

## 2.1 Importing and entry processing

In [14]:
train_set = pd.read_csv("../data/train.csv") # raw data

# DV
Y_train = train_set["label"]
# IV
X_train = train_set.drop(labels = ["label"], axis = 1)
#X_train

## 2.2Normalization to [0-1]  range

In [15]:
X_train = X_train / 255


X_train = X_train.values.reshape(-1,28,28,1)

Y_train = to_categorical(Y_train, num_classes = 10)

## 2.3Data spliting into train & valid set 

In [16]:
X_train, X_valid , Y_train , Y_valid = train_test_split(X_train, Y_train, test_size = 0.2, random_state = 42)

## 2.4 Augmentation

In [17]:
igd = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    rotation_range=10,
    zoom_range = 0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=False,
    vertical_flip=False
)

igd.fit(X_train)

# 3. Building the architecture of Keras model & Compilation

## 3.1 Architecture

In [18]:
model = Sequential(
        [
            Conv2D(32, (3,3),activation = "relu", input_shape = (28,28,1)),
            MaxPool2D(pool_size = (2, 2)),
            Conv2D(32, (3,3),activation = "relu"),
            MaxPool2D(pool_size = (2, 2)),
            Flatten(),
            Dropout(0.25),
            Dense(units = 32, activation = "relu"),
            Dense(units = 10, activation = "sigmoid")  
        ]
)

## 3.2 Compilation

In [19]:
model.compile(loss='categorical_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])

# 4. Training

In [ ]:
model.fit_generator(
    igd.flow(X_train, Y_train, batch_size = 32),
    epochs = 10,
    steps_per_epoch = X_train.shape[0] // 32,
    validation_data = (X_valid, Y_valid)
    )

Epoch 1/10
1050/1050 [==============================] - 32s 30ms/step - loss: 0.5945 - acc: 0.8104 - val_loss: 0.1341 - val_acc: 0.9631
Epoch 2/10
1050/1050 [==============================] - 30s 29ms/step - loss: 0.2316 - acc: 0.9299 - val_loss: 0.0695 - val_acc: 0.9796
Epoch 3/10
1050/1050 [==============================] - 34s 32ms/step - loss: 0.1738 - acc: 0.9470 - val_loss: 0.0601 - val_acc: 0.9821: 0.1746 -
Epoch 4/10
1050/1050 [==============================] - 32s 31ms/step - loss: 0.1446 - acc: 0.9545 - val_loss: 0.0502 - val_acc: 0.9846: 7s - - ETA: 1s -
Epoch 5/10
1050/1050 [==============================] - 32s 31ms/step - loss: 0.1280 - acc: 0.9607 - val_loss: 0.0691 - val_acc: 0.9781
Epoch 6/10
1050/1050 [==============================] - 33s 31ms/step - loss: 0.1118 - acc: 0.9662 - val_loss: 0.0441 - val_acc: 0.9862- ETA: 0s - loss: 0.1115 - acc: 0.
Epoch 7/10
1050/1050 [==============================] - 31s 30ms/step - loss: 0.1034 - acc: 0.9685 - val_loss: 0.0419 - va

# Saving a model

In [12]:
tfjs.converters.save_keras_model(model, './ModelJS')